In [ ]:
import pyaudio  # recording and running an audio file
import whisper  # for speech2text
import wave     # to write the audio as WAV file
import os       # file management
import openai   # for GPT3 
import requests # to handle timeout of the API call

## Record and Save Audio File:

In [ ]:
# Parameters for Recording from microphone
CHANNELS = 1    # mono/stereo
FORMAT = pyaudio.paInt16
RATE = 44100    # Hz
CHUNK = 1024    
TEMP_AUDIO_FILE_PATH = "./temp.wav"
DURATION = 20    # seconds of recording



In [ ]:
# RECORD FROM MICROPHONE
p = pyaudio.PyAudio()
stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)
print("Recording in progress...")
frames = [] # list of all audio chunks.

# Each chunk is 1024 samples.
# so every second we get 44100 / 1024 ~ 43 chunks.
# so for 5 seconds, we have to record 43*5 chunks.

for i in range(0, int(RATE / CHUNK * DURATION)):
    data = stream.read(CHUNK)
    frames.append(data)

stream.stop_stream()
print("Recording stopped.")
stream.close()
p.terminate()

print("Writing Audio File...")
with wave.open(TEMP_AUDIO_FILE_PATH, 'wb') as wf:
    wf.setparams((CHANNELS, p.get_sample_size(FORMAT), RATE, 0, 'NONE', 'NONE'))
    wf.writeframes(b''.join(frames))
    wf.close()
print(f"Audio File Written: {TEMP_AUDIO_FILE_PATH}")

### Audio Example:
For example, set the `DURATION` to 20 seconds and read this out:

"If, however, the existence of evil is to be consistently explained as a necessary condition for the realization of certain kinds of good, then the kind of good that requires the existence of evil must be identified and shown to be such that it could not be realized in a world without evil."

## Transcribe the audio

In [ ]:
model = whisper.load_model("base") # transcribes the audio file
result = model.transcribe(TEMP_AUDIO_FILE_PATH)
text = result["text"]
print(f"Transcription of {TEMP_AUDIO_FILE_PATH} :")
print(text)

## Summarize the transcription

[Prompt engineering is an important aspect of interacting with Large Language Models]

In [ ]:
openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:

# prompt = (f"Summarize this text: {text}")
# prompt = (f"Summarize this text in fewer words: {text}")
# prompt = (f"Summarize this text in fewer and simpler words: {text}")
prompt = (f"Explain this text in simpler words: {text}")

# Call GPT3
try:
    openai_output = openai.Completion.create(
        engine = "text-davinci-003",
        prompt=prompt,
        max_tokens=1024,
        n=1,
        stop=None,
        temperature=0.6,
        timeout=10
        )
    print("openAI.completion created")
except requests.exceptions.ReadTimeout:
    print("Timeout: openAI.Completion took too long to respond")
    openai_output = None

In [ ]:
# Get summary
summary = openai_output.choices[0].text

print("Summary/Explanation:")
print(summary)